In [1]:
import os
import tarfile
import requests

tar_url = "https://pub-25c498004d1e4d4c8da69b2c05676836.r2.dev/super_resolution_val_set_2024-01-09.tar"
tar_filename = "data.tar"

# Download
response = requests.get(tar_url, stream=True)
with open(tar_filename, "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        if chunk:
            f.write(chunk)

# Check file type
with open(tar_filename, "rb") as f:
    start = f.read(4)
    print("File signature:", start)

# Try different tarfile modes
extract_dir = "./extracted_data"
os.makedirs(extract_dir, exist_ok=True)

try:
    with tarfile.open(tar_filename, "r") as tar:
        tar.extractall(path=extract_dir)
        print("Extracted as plain tar")
except tarfile.ReadError:
    try:
        with tarfile.open(tar_filename, "r:gz") as tar:
            tar.extractall(path=extract_dir)
            print("Extracted as gzip tar")
    except tarfile.ReadError:
        try:
            with tarfile.open(tar_filename, "r:xz") as tar:
                tar.extractall(path=extract_dir)
                print("Extracted as xz tar")
        except tarfile.ReadError:
            print("Could not extract: not a valid tar file or wrong mode.")

File signature: b'val_'
Extracted as plain tar


In [2]:
import os
import tarfile
import requests

tar_url = "https://pub-25c498004d1e4d4c8da69b2c05676836.r2.dev/super_resolution_small_val_set_2024-01-09.tar"
tar_filename = "data_val.tar"

# Download
response = requests.get(tar_url, stream=True)
with open(tar_filename, "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        if chunk:
            f.write(chunk)

# Check file type
with open(tar_filename, "rb") as f:
    start = f.read(4)
    print("File signature:", start)

# Try different tarfile modes
extract_dir = "./extracted_data"
os.makedirs(extract_dir, exist_ok=True)

try:
    with tarfile.open(tar_filename, "r") as tar:
        tar.extractall(path=extract_dir)
        print("Extracted as plain tar")
except tarfile.ReadError:
    try:
        with tarfile.open(tar_filename, "r:gz") as tar:
            tar.extractall(path=extract_dir)
            print("Extracted as gzip tar")
    except tarfile.ReadError:
        try:
            with tarfile.open(tar_filename, "r:xz") as tar:
                tar.extractall(path=extract_dir)
                print("Extracted as xz tar")
        except tarfile.ReadError:
            print("Could not extract: not a valid tar file or wrong mode.")

File signature: b'smal'
Extracted as plain tar


In [3]:
!mv /kaggle/working/extracted_data/val_set /kaggle/working/extracted_data/train_set
!mv /kaggle/working/extracted_data/small_val_set /kaggle/working/extracted_data/val_set

In [4]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm

def normalize_and_save(img_path, out_path, norm_factor, clip_max=None):
    img = Image.open(img_path)
    arr = np.array(img).astype(np.float32)
    arr = arr / norm_factor
    if clip_max is not None:
        arr = np.clip(arr, 0, clip_max)
    arr = (arr * 255).astype(np.uint8)  # Save as 8-bit PNG for compatibility
    Image.fromarray(arr).save(out_path)

def process_sentinel2(s2_dir, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    for tile in tqdm(os.listdir(s2_dir), desc="Sentinel-2 tiles"):
        tile_path = os.path.join(s2_dir, tile)
        out_tile_path = os.path.join(out_dir, tile)
        os.makedirs(out_tile_path, exist_ok=True)
        for band_file in os.listdir(tile_path):
            band_path = os.path.join(tile_path, band_file)
            out_band_path = os.path.join(out_tile_path, band_file)
            if band_file == "tci.png":
                normalize_and_save(band_path, out_band_path, 255)
            else:
                normalize_and_save(band_path, out_band_path, 8160, 1.0)

def process_naip(naip_dir, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    for uuid in tqdm(os.listdir(naip_dir), desc="NAIP uuids"):
        uuid_path = os.path.join(naip_dir, uuid)
        out_uuid_path = os.path.join(out_dir, uuid)
        os.makedirs(out_uuid_path, exist_ok=True)
        for tile in os.listdir(uuid_path):
            tile_path = os.path.join(uuid_path, tile)
            out_tile_path = os.path.join(out_uuid_path, tile)
            os.makedirs(out_tile_path, exist_ok=True)
            rgb_path = os.path.join(tile_path, "rgb.png")
            out_rgb_path = os.path.join(out_tile_path, "rgb.png")
            if os.path.exists(rgb_path):
                normalize_and_save(rgb_path, out_rgb_path, 255)

# Example usage:
process_sentinel2("./extracted_data/train_set/sentinel2", "./extracted_data/train_set/sentinel2_norm")
process_naip("./extracted_data/train_set/naip", "./extracted_data/train_set/naip_norm")
# Repeat for val_set as needed

NAIP uuids: 100%|██████████| 7688/7688 [00:36<00:00, 210.34it/s]


In [ ]:
# import rasterio
# from rasterio.warp import reproject

# tci_jp2_path = '/kaggle/working/extracted_data/val_set/sentinel2/20771_46525/tci.png'
# with rasterio.open(tci_jp2_path) as src:
#     img_rep, meta_rep = reproject(
#         img, meta, rasterio.crs.CRS.from_epsg(3857), resolution=(9.555, 9.555), resampling=rasterio.warp.Resampling.bilinear)

# with rasterio.open(tci_jp2_path.replace('.jp2', '_rep.jp2', 'w', **meta_rep) as dst:
#     dst.write(img_rep)

In [9]:
import os
import shutil
import random
from pathlib import Path
from tqdm import tqdm

# Define paths
input_root = Path("/kaggle/working/extracted_data/train_set")
output_root = Path("/kaggle/working/dataset")

naip_dir = input_root / "naip_norm"
s2_dir = input_root / "sentinel2_norm"

train_hr_dir = output_root / "train" / "HR"
train_lr_dir = output_root / "train" / "LR"
val_hr_dir = output_root / "val" / "HR"
val_lr_dir = output_root / "val" / "LR"

# Create output folders
for d in [train_hr_dir, train_lr_dir, val_hr_dir, val_lr_dir]:
    d.mkdir(parents=True, exist_ok=True)

# Gather all HR-LR pairs
pairs = []

for uuid_folder in naip_dir.glob("*"):
    for tile_folder in uuid_folder.glob("*"):
        hr_path = tile_folder / "rgb.png"
        tile_name = tile_folder.name
        lr_path = s2_dir / tile_name / "tci.png"
        if hr_path.exists() and lr_path.exists():
            pairs.append((hr_path, lr_path, tile_name))

# Shuffle and split into train/val
random.seed(42)
random.shuffle(pairs)
split_idx = int(len(pairs) * 0.8)
train_pairs = pairs[:split_idx]
val_pairs = pairs[split_idx:]

# Helper function
def copy_pairs(pairs, hr_out_dir, lr_out_dir):
    for idx, (hr, lr, tile) in enumerate(tqdm(pairs, desc=f"Copying to {hr_out_dir.parent.name}")):
        name = f"{tile}_{idx:04d}.png"
        shutil.copy(hr, hr_out_dir / name)
        shutil.copy(lr, lr_out_dir / name)

# Copy files
copy_pairs(train_pairs, train_hr_dir, train_lr_dir)
copy_pairs(val_pairs, val_hr_dir, val_lr_dir)

print("✅ Dataset conversion complete.")


Copying to val: 100%|██████████| 1639/1639 [00:01<00:00, 1490.40it/s]

✅ Dataset conversion complete.


In [ ]:
def check_image_sizes(img_path, expected_shape):
    img = Image.open(img_path)
    if img.size != expected_shape:
        print(f"Size mismatch: {img_path} is {img.size}, expected {expected_shape}")

# Example:
check_image_sizes("/kaggle/working/extracted_data/val_set/sentinel2/20771_46525/tci.png", (32, 32))
check_image_sizes("/kaggle/working/extracted_data/val_set/naip/m_2608053_sw_17_060_20191203/36226_55652/rgb.png", (128, 128))

In [38]:
from PIL import Image
import os
from pathlib import Path
from tqdm import tqdm

# Set root directory
dataset_root = Path("/kaggle/working/dataset")

# Tile size for LR
tile_size = 32

# Apply to both train and val LR folders
for split in ["train", "val"]:
    lr_dir = dataset_root / split / "LR"
    print(f"🔄 Processing {lr_dir}")
    
    for img_path in tqdm(sorted(lr_dir.glob("*.png"))):
        img = Image.open(img_path)
        w, h = img.size
        base_name = img_path.stem

        # Crop into non-overlapping 32x32 tiles
        count = 0
        for top in range(0, h - tile_size + 1, tile_size):
            for left in range(0, w - tile_size + 1, tile_size):
                tile = img.crop((left, top, left + tile_size, top + tile_size))
                tile_name = f"{base_name}_tile_{top}_{left}.png"
                tile.save(lr_dir / tile_name)
                count += 1

        # Remove original file
        img_path.unlink()

        print(f"✅ Cropped {base_name}.png into {count} tiles.")

print("🎉 All LR images cropped in-place to 32×32 tiles.")


🔄 Processing /kaggle/working/dataset/train/LR


  0%|          | 4/6553 [00:00<03:06, 35.17it/s]

✅ Cropped 20197_45697_1483.png into 32 tiles.
✅ Cropped 20213_45762_0577.png into 32 tiles.
✅ Cropped 20221_45760_5680.png into 32 tiles.
✅ Cropped 20298_45953_1867.png into 32 tiles.
✅ Cropped 20303_45457_0437.png into 32 tiles.


  0%|          | 9/6553 [00:00<02:40, 40.76it/s]

✅ Cropped 20321_45566_1108.png into 32 tiles.
✅ Cropped 20323_46125_2457.png into 32 tiles.
✅ Cropped 20325_45526_3468.png into 32 tiles.
✅ Cropped 20329_45445_4697.png into 32 tiles.


...

✅ Cropped 38422_49220_1549.png into 32 tiles.
✅ Cropped 38452_49413_1037.png into 32 tiles.
✅ Cropped 38504_48695_0319.png into 32 tiles.
✅ Cropped 38508_48570_1330.png into 32 tiles.
✅ Cropped 38508_49256_0101.png into 32 tiles.
✅ Cropped 38535_48580_0072.png into 32 tiles.
✅ Cropped 38550_49628_1585.png into 32 tiles.
✅ Cropped 38555_49093_0700.png into 32 tiles.
✅ Cropped 38558_48343_0785.png into 32 tiles.
✅ Cropped 38567_49287_0968.png into 32 tiles.
✅ Cropped 38588_49199_0264.png into 32 tiles.


100%|█████████▉| 1635/1639 [00:33<00:00, 55.77it/s]

✅ Cropped 38593_49497_1230.png into 32 tiles.
✅ Cropped 38603_48789_1150.png into 32 tiles.
✅ Cropped 38615_48827_0532.png into 32 tiles.
✅ Cropped 38699_47804_0783.png into 32 tiles.
✅ Cropped 38700_48505_0195.png into 32 tiles.
✅ Cropped 38701_48113_0342.png into 32 tiles.
✅ Cropped 38701_48352_1268.png into 32 tiles.
✅ Cropped 38704_48601_0255.png into 32 tiles.
✅ Cropped 38707_48705_1441.png into 32 tiles.
✅ Cropped 38723_48450_0029.png into 32 tiles.
✅ Cropped 38725_47349_0182.png into 32 tiles.
✅ Cropped 38739_47173_0794.png into 32 tiles.
✅ Cropped 38761_47976_1210.png into 32 tiles.


100%|██████████| 1639/1639 [00:33<00:00, 48.51it/s]

✅ Cropped 38770_48499_0265.png into 32 tiles.
✅ Cropped 38785_47489_1323.png into 32 tiles.
✅ Cropped 38819_47373_1340.png into 32 tiles.
✅ Cropped 38839_49324_1591.png into 32 tiles.
🎉 All LR images cropped in-place to 32×32 tiles.


In [ ]:
def check_image_sizes(img_path, expected_shape):
    img = Image.open(img_path)
    if img.size != expected_shape:
        print(f"Size mismatch: {img_path} is {img.size}, expected {expected_shape}")

# Example:
check_image_sizes("/kaggle/working/extracted_data/val_set/sentinel2/20771_46525/tci.png", (32, 32))
check_image_sizes("/kaggle/working/extracted_data/val_set/naip/m_2608053_sw_17_060_20191203/36226_55652/rgb.png", (128, 128))

In [14]:
!git clone https://github.com/ParamAhuja/Real_ESRGAN.git

Cloning into 'Real_ESRGAN'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 113 (delta 14), reused 110 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (113/113), 4.83 MiB | 15.71 MiB/s, done.
Resolving deltas: 100% (14/14), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 794.7 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 3.0 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.6/299.6 kB 5.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 17.6 MB/s eta 0:00:

In [35]:
!pip install -q -r /kaggle/working/Real_ESRGAN/requirements.txt

In [31]:
# !cd /kaggle/working/Real_ESRGAN/
# !apt-get install -y gdal-bin python3-gdal

/kaggle/working


In [30]:
# !pwd

/kaggle/working


In [17]:
# get the data and preprocess ------------------------- done
# ensure the correct format and structure of data ----- done
# !!!!!problem - size mismatch of tci - sentinel2 -----
# clone repo ------------------------------------------ done
# install reqs ---------------------------------------- done
# get the model --------------------------------------- done
# ensure correct placement of model ------------------- done
# get and place metadata file ------------------------- done
# edit the configs ------------------------------------ 
# start finetune -------------------------------------- 

In [18]:
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models

--2025-07-24 12:56:29--  https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/387326890/08f0e941-ebb7-48f0-9d6a-73e87b710e7e?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-07-24T13%3A43%3A17Z&rscd=attachment%3B+filename%3DRealESRGAN_x4plus.pth&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-07-24T12%3A43%3A02Z&ske=2025-07-24T13%3A43%3A17Z&sks=b&skv=2018-11-09&sig=e%2FQdVKLKWDG8G93U3vGFcO4UVYY%2BYymng7uu8O%2F6QmY%3D&jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1MzM2MjA4OSwibmJmIjoxNzUzMzYxNzg5LCJwYXRoIjoicmVsZWFzZWFzc2

In [32]:
!python /kaggle/working/Real_ESRGAN/scripts/generate_meta_info_pairdata.py --input /kaggle/working/dataset/train/HR /kaggle/working/dataset/train/LR --meta_info /kaggle/working/dataset/train/meta_info/meta_info_DIV2K_sub_pair.txt > /dev/null
# !python /kaggle/working/Real_ESRGAN/scripts/generate_meta_info_pairdata.py --input /kaggle/working/dataset/train/HR /kaggle/working/dataset/train/LR --meta_info /kaggle/working/dataset/train/meta_info/meta_info_DIV2K_sub_pair.txt 

HR/20197_45697_1483.png, LR/20197_45697_1483.png
HR/20213_45762_0577.png, LR/20213_45762_0577.png
HR/20221_45760_5680.png, LR/20221_45760_5680.png
HR/20298_45953_1867.png, LR/20298_45953_1867.png
HR/20303_45457_0437.png, LR/20303_45457_0437.png
HR/20321_45566_1108.png, LR/20321_45566_1108.png
HR/20323_46125_2457.png, LR/20323_46125_2457.png
HR/20325_45526_3468.png, LR/20325_45526_3468.png
HR/20329_45445_4697.png, LR/20329_45445_4697.png
HR/20763_46106_6098.png, LR/20763_46106_6098.png
HR/20763_46195_6200.png, LR/20763_46195_6200.png
HR/20764_45315_0555.png, LR/20764_45315_0555.png
HR/20766_46063_2703.png, LR/20766_46063_2703.png
HR/20772_45146_1571.png, LR/20772_45146_1571.png
HR/20775_46483_4943.png, LR/20775_46483_4943.png
HR/20781_46147_4230.png, LR/20781_46147_4230.png
HR/20799_45850_2790.png, LR/20799_45850_2790.png
HR/20805_45183_6220.png, LR/20805_45183_6220.png
HR/20812_45810_4381.png, LR/20812_45810_4381.png
HR/20837_46171_2529.png, LR/20837_46171_2529.png
HR/20838_45638_0270.

In [36]:
!python /kaggle/working/Real_ESRGAN/realesrgan/train.py -opt /kaggle/working/Real_ESRGAN/options/finetune_realesrgan_x4plus_pairdata.yml --auto_resume

Traceback (most recent call last):
  File "/kaggle/working/Real_ESRGAN/realesrgan/train.py", line 3, in <module>
    from basicsr.train import train_pipeline
  File "/usr/local/lib/python3.11/dist-packages/basicsr/__init__.py", line 4, in <module>
    from .data import *
  File "/usr/local/lib/python3.11/dist-packages/basicsr/data/__init__.py", line 22, in <module>
    _dataset_modules = [importlib.import_module(f'basicsr.data.{file_name}') for file_name in dataset_filenames]
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/basicsr/data/__init__.py", line 22, in <listcomp>
    _dataset_modules = [importlib.import_module(f'basicsr.data.{file_name}') for file_name in dataset_filenames]
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[